<div class="alert alert-info">
<h3>Задание</h3>
Основные цели этого задания:

- Научиться генерировать негативы.
- Научиться настраивать алгоритмы коллаборативной фильтрации.

<h4>Задача:</h4>
Научиться рекомендовать пользователям фильмы на основе факта просмотра фильмов пользователями. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from random import choice, random
import scipy.sparse as sp
from sklearn.metrics.pairwise import cosine_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds

pd.options.display.float_format ='{:,.3f}'.format

## <div style="border: 1px solid purple; padding: 10px; color: blue"> Задача 1.</div>
Предположим, постановка рейтинга — обязательное по итогам просмотра фильмов действие. Основываясь на этом, сгенерируйте новый целевой признак «факт просмотра фильма пользователем», который будет равен 1 для всех пар пользователь * фильм из подгруженного датасета.

In [2]:
try:
    ratings = pd.read_csv("ratings_df_sample_2.csv")
    movies = pd.read_csv("movies.csv")
except FileNotFoundError:
    print("You have to load the file to the directory before opening it.")

In [3]:
ratings.head(5)

,userId,movieId,rating,timestamp
0,54,2,3.000,974918176
1,54,32,5.000,974836809
2,54,47,4.000,974837760
3,54,50,4.000,974837760
4,54,223,5.000,974840217


In [4]:
len_df  = ratings.shape

In [5]:
print(f"Ratings {len_df}, movies {movies.shape}")

Ratings (6040099, 4), movies (27278, 3)


In [6]:
ratings["is_viewed"] = 1

In [7]:
ratings.is_viewed.value_counts()

is_viewed
1    6040099
Name: count, dtype: int64

In [8]:
ratings = ratings.merge(movies, on="movieId")

In [9]:
ratings.describe()

,userId,movieId,rating,timestamp,is_viewed
count,"6,040,099.000","6,040,099.000","6,040,099.000","6,040,099.000","6,040,099.000"
mean,"68,804.817","4,822.958",3.554,"1,115,774,334.985",1.000
std,"40,102.241","11,368.034",1.003,"135,843,321.295",0.000
min,7.000,1.000,0.500,"824,835,410.000",1.000
25%,"34,180.000",919.000,3.000,"995,660,158.000",1.000
50%,"68,914.000","1,876.000",4.000,"1,111,706,240.000",1.000
75%,"103,281.000","3,448.000",4.000,"1,213,151,458.500",1.000
max,"138,493.000","81,845.000",5.000,"1,427,780,469.000",1.000


Перекодируем айди фильма и пользователя, чтобы они шли по порядку.

In [10]:
ratings["userId"] = pd.factorize(ratings["userId"])[0]
ratings["movieId"] = pd.factorize(ratings["movieId"])[0]

In [11]:
ratings.movieId.min(), ratings.movieId.max()

(0, 999)

## <div style="border: 1px solid purple; padding: 10px; color: blue"> Задача 2.</div>
А откуда взять «нолики»? В наших данных есть только пары пользователь * фильм, в которых пользователь точно смотрел фильм. Но для обучения модели нужны так называемые «негативы», то есть, пары, где пользователь фильм не смотрел. На практике приходится сталкиваться с необходимостью генерировать их вручную, давайте потренируемся это делать.

2.1. Сначала найдите уникальные id всех пользователей и уникальные id всех фильмов.

In [12]:
unique_users = ratings.userId.unique()
unique_films = ratings.movieId.unique()
print(
    f"Unique users: {unique_users.shape[0]}\n\
unique films: {unique_films.shape[0]}"
)

Unique users: 20000
unique films: 1000


2.2. С помощью функции random.choice (документация) сгенерируйте случайные пары пользователь * фильм\
2.3. Поскольку среди сгенерированных пар могут быть и такие, что пользователь в них уже смотрел фильм, сгенерируйте побольше пар, например, удвоенное количество строк из источника. Это может занять пару минут.

In [13]:
random_pairs = pd.DataFrame(
    [
        (
            choice(range(0, unique_users.shape[0])),
            choice(range(0, unique_films.shape[0])),
        )
        for i in range(ratings.shape[0] * 2)
    ],
    columns=["userId", "movieId"],
)

2.4. Среди сгенерированных пар могут быть и дубликаты, удалите их.

In [14]:
print(f'Duplicates sum: {random_pairs[random_pairs.duplicated()].shape[0]}')
random_pairs.drop_duplicates(inplace=True, ignore_index=True)

Duplicates sum: 3013290


2.5. Оставьте среди сгенерированных пар только те, в которых пользователь фильм не смотрел.\
2.6. Возможно, пар получилось больше, чем нужно, выберите из них столько, сколько у нас строк в исходных данных.\
2.7. Добавьте очищенные сгенерированные пары к исходным данным. Значение целевого признака в них будет равно нулю. Убедитесь, что у вас не появились дубликаты в датасете.

Соединим данные, таким образом, быстрее уберем дубликаты по столбцам с айди пользователя и фильма. 

In [15]:
ratings = pd.concat([ratings,random_pairs])
ratings.shape

(15107007, 7)

Удалим дубликаты по столбцам с айди, оставляя первый дубликат (так как первыми идут исходные данные)

In [16]:
ratings.drop_duplicates(
    inplace=True, ignore_index=True, subset=["userId", "movieId"], keep="first"
)

In [17]:
ratings.shape

(12371088, 7)

In [18]:
ratings = ratings.iloc[:(len_df[0] * 2)]
ratings.shape

(12080198, 7)

In [19]:
print(f'Duplicates sum: {ratings[ratings.duplicated()].shape[0]}')

Duplicates sum: 0


Заполним нулями новые пары пользователь-фильм.

In [20]:
ratings.is_viewed = ratings.is_viewed.fillna(0)

In [21]:
ratings.sample(3)

,userId,movieId,rating,timestamp,is_viewed,title,genres
7104358,9026,57,NaN,NaN,0.000,NaN,NaN
2150869,6183,431,4.500,"1,348,686,378.000",1.000,Interview with the Vampire: The Vampire Chroni...,Drama|Horror
7223874,18222,687,NaN,NaN,0.000,NaN,NaN


## <div style="border: 1px solid purple; padding: 10px; color: blue"> Задача 3.</div>
Подготовьте датасет к обучению: отделите тестовую часть от тренировочной.

In [22]:
train_data, test_data = train_test_split(ratings, test_size=0.01)

## <div style="border: 1px solid purple; padding: 10px; color: blue"> Задача 4.</div>
Обучите dummy-model. Пусть она будет возвращать случайную вероятность принадлежности классу 1. Для этого можете использовать функцию random.random (документация). Оцените ее качество какой-то метрикой на свой вкус. Необходимо прогнозировать именно вероятность, чтобы была возможность ранжировать по ней варианты для рекомендации лучшего контента пользователю.

In [23]:
y_random = [random() for i in range(test_data.shape[0])]

In [24]:
test_data["predict_random"] = y_random

In [25]:
test_data.head()

,userId,movieId,rating,timestamp,is_viewed,title,genres,predict_random
4749911,14592,167,3.000,"938,746,784.000",1.000,Jerry Maguire (1996),Drama|Romance,0.873
10391579,6349,351,NaN,NaN,0.000,NaN,NaN,0.828
11117371,18787,554,NaN,NaN,0.000,NaN,NaN,0.361
7328153,8565,780,NaN,NaN,0.000,NaN,NaN,0.624
6538188,17083,750,NaN,NaN,0.000,NaN,NaN,0.079


Для оценки моделей используется метрика RMSE, так как она лучше подходит для сравнения качества моделей.

In [26]:
rmse = np.sqrt(mean_squared_error(test_data["predict_random"], test_data["is_viewed"]))
print(f"RMSE: {rmse}")

RMSE: 0.5782428510510979


In [27]:
results = {}
results["random"] = rmse

## <div style="border: 1px solid purple; padding: 10px; color: blue"> Задача 5.</div>
Реализуйте три алгоритма коллаборативной фильтрации: user-, item-based и алгоритм на основе матричной факторизации. Оцените их качество и адекватность. Если качество недостаточно хорошее, попробуйте варьировать параметры: количество похожих пользователей/фильмов, количество элементов в матрицах при матричном разложении.

Создадим матрицу user-item. В случае, если нет информации смотрел ли пользователь фильм или нет, возьмем случайные вероятности.

In [28]:
user_item_matrix = np.random.random(
    size=(train_data.userId.nunique(), train_data.movieId.nunique())
)

Для разложения матрицы необходимо, чтобы тип значений в ней был float.

In [29]:
for row in train_data.to_dict(orient="records"):
    user_item_matrix[row["userId"], row["movieId"]] = float(row["is_viewed"])

In [30]:
user_item_matrix

array([[1.        , 1.        , 1.        , ..., 0.        , 0.12244168,
        0.67378753],
       [1.        , 1.        , 1.        , ..., 0.04939297, 0.        ,
        0.        ],
       [1.        , 1.        , 1.        , ..., 0.34493458, 0.        ,
        0.56101566],
       ...,
       [0.        , 0.92311035, 0.        , ..., 0.        , 0.        ,
        0.73940459],
       [0.        , 0.        , 0.48870682, ..., 0.27365426, 1.        ,
        0.        ],
       [0.72280199, 0.        , 0.        , ..., 0.96493973, 0.99673084,
        0.        ]])

### Алгоритм user-based

Определим количество схожих фильмов и пользователей.

In [31]:
top=10

In [32]:
user_similarity = cosine_distances(user_item_matrix)

In [33]:
top_similar_users = []
for i in range(train_data.userId.nunique()):
    neighbors = (user_similarity[i]).argsort()[1 : top + 1]
    top_similar_users.append(user_item_matrix[neighbors])
top_similar_users = np.array(top_similar_users)

Возьмем среднее значение по ближайшим пользователям.

In [34]:
pred_user_based = top_similar_users.mean(axis=1)

In [35]:
test_data["predict_user_based"] = test_data.apply(
    lambda f: pred_user_based[f["userId"], f["movieId"]], axis=1
)

In [36]:
rmse = np.sqrt(
    mean_squared_error(test_data["predict_user_based"], test_data["is_viewed"])
)
print(f"RMSE: {rmse}")

RMSE: 0.6561431096712502


In [37]:
results['predict_user_based'] = rmse

### Алгоритм item-based

In [38]:
movies_similarity = cosine_distances(user_item_matrix.T)

In [39]:
top_similar_ratings = []
for i in range(train_data.movieId.nunique()):
    neighbors = (movies_similarity[i]).argsort()[1 : top + 1]
    top_similar_ratings.append(user_item_matrix.T[neighbors])

top_similar_ratings = np.array(top_similar_ratings)

In [40]:
pred_item_based = top_similar_ratings.mean(1).T

In [41]:
test_data["predict_item_based"] = test_data.apply(
    lambda f: pred_item_based[f["userId"], f["movieId"]], axis=1
)

In [42]:
rmse = np.sqrt(
    mean_squared_error(test_data["predict_item_based"], test_data["is_viewed"])
)
print(f"RMSE: {rmse}")

RMSE: 0.6004838915132379


In [43]:
results["predict_item_based"] = rmse

### Алгоритм на основе матричного разложения

In [44]:
u, s, vh = svds(user_item_matrix, k=top)

In [45]:
users = np.dot(u, np.diag(s))
items = vh.T

In [46]:
np.diag(s).shape

(10, 10)

In [47]:
test_data['svd_predictions'] = test_data.apply(
    lambda f: np.dot(users[f['userId']], items[f['movieId']]), axis = 1
)u

In [48]:
rmse = np.sqrt(mean_squared_error(test_data["svd_predictions"], test_data["is_viewed"]))
print(f"RMSE: {rmse}")

RMSE: 0.4060473050163802


In [49]:
results["svd_predictions"] = rmse

## <div style="border: 1px solid purple; padding: 10px; color: blue"> Задача 6.</div>
Опишите вывод, содержащий информацию о том, какой алгоритм проявил себя лучше всего.

In [50]:
pd.DataFrame.from_dict(
    results, orient="index", columns=["result"]
).reset_index().sort_values(by="result")

,index,result
3,svd_predictions,0.406
0,random,0.578
2,predict_item_based,0.600
1,predict_user_based,0.656


<div class="alert alert-info">
<h3>Вывод</h3>

Лучше всех проявил себя алгоритм на основе матричного разложения.